Import tools & libraries

In [ ]:
# importing python package
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
#import math
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as mse 
#from sklearn.metrics import mean_absolute_error as mae

Objective: To predict the price of a house based on features provided in the dataset.
Get contents of the Data (csv or excel file)

In [ ]:
df = pd.read_excel('C:/Users/ronal/Documents/IRONHACK/data_mid_bootcamp_project_regression/regression_data.xls')

In [ ]:
df1 =df.copy(deep = True)

Exploration of Data (EDA)
- check for structure of the data
- # headers are standarized on format
- check numerical variable visually

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head(5)

In [ ]:
df.hist(figsize=(15,15));

Check every column and deal with odd or `NaN` values
Some properties have been sold more than one time (176), we are dropping those values.

In [ ]:
df.isnull().values.any()

In [ ]:
x = df.isna().sum()/len(df)*100
x

In [ ]:
for x in df.columns:
    print(x, len(df[x].unique()))

In [ ]:
df['id'].value_counts()

In [ ]:
(df['id'].value_counts()>1).sum()

fixing data types for some features & some data cleaning.

In [ ]:
df['condition'] = df['condition'].astype('object')
df['grade'] = df['grade'].astype('object')

In [ ]:
df['waterfront'].unique()

In [ ]:
df['waterfront'].value_counts()

values to be dropped 
waterfront (boolean) - 99% no waterfront
view (boolean) - 90% no view
bedrooms less than 9 not significant

In [ ]:
data2 = df[df['bedrooms'] < 9]

In [ ]:
data2.duplicated(subset=None, keep= False )

In [ ]:
data2.drop_duplicates()

In [ ]:
data2[data2.duplicated(keep = False)]

we don't need id nor date for our analysis, we also dopping waterfront & view here (eliminating bias on Data)

In [ ]:
data2 = data2.drop(['id', 'date', 'zipcode', 'waterfront', 'view'], axis = 1, inplace = False)

checking again for data types and nulls values
Which columns are numerical, which are categoricals

In [ ]:
data2.dtypes

In [ ]:
data2.info()

In [ ]:
visualization on categoricals

In [ ]:
# condition vs price
sns.barplot(data = data2, x = 'condition', y ='price');

In [ ]:
# grade vs price
sns.barplot(data = data2, x = 'grade', y ='price');

I choose to do some transformation to use this data
yr_built  - how long have been built
yr_renovated - how long since last renovation

In [ ]:
data2['yr_built'].nunique

In [ ]:
data2['yr_renovated'].nunique

In [ ]:
max(data2['yr_built'])

built a funtion to modify fiscal year into amount of years, since we have some values 0 on yr_renovated we're going 
to use in those cases the yr_built for the purpose of this task then passing the funtion to modifiers

In [ ]:
def modify (built, renov):
    if renov == 0:
        return built
    else:
        return renov

In [ ]:
yrs_modif = list(map(modify, data2['yr_built'], data2['yr_renovated']))

In [ ]:
data2['yr_renovated'] = yrs_modif

In [ ]:
data2['yr_built'] = data2['yr_built'].sub(2015).mul(-1)

In [ ]:
data2['yr_renovated'] = data2['yr_renovated'].sub(2015).mul(-1)

In [ ]:
data2

Explore the data, look for correlations and patterns
merge x and y
describe chart (.describe().t)
IQR

In [ ]:
correlations_matrix = data2.corr()
mask = np.zeros_like(correlations_matrix)
mask[np.triu_indices_from(mask)] = True

fig, ax = plt.subplots(figsize = (20,15))
ax = sns.heatmap(correlations_matrix, mask = mask, annot= True, cmap='Blues')
plt.yticks(rotation = 0)
plt.show()

In [ ]:
correlations_matrix.shape

In [ ]:
data2.describe().T

looking for outliers here and eliminating

In [ ]:
for x in data2.columns:
    if data2[x].dtype == "int64" or data2[x].dtype == "float64":
        sns.distplot(data2[x])
        plt.show()

data needing log transf, taking care of some outliers here

In [ ]:
list2 = ['sqft_lot', 'sqft_above','sqft_basement', 'sqft_lot15']

In [ ]:
lost_data = 0
for x in list2:
    o_size = data2.shape[0]
    iqr = np.percentile(data2[x],75) - np.percentile(data2[x],25)
    upper_limit = np.percentile(data2[x],75) + 1.5*iqr
    lower_limit = np.percentile(data2[x],25) - 0.5*iqr
    gk_log = data2[(data2[x]>lower_limit) & (data2[x]<upper_limit)][x]
    lost_data += o_size - gk_log.shape[0]
    sns.distplot(gk_log)
    plt.show()
    
# iqr = np.percentile(data1['sqft_lot'],75) - np.percentile(data1['sqft_lot'],25)
# upper_limit = np.percentile(data1['sqft_lot'],75) + 1.5*iqr
# lower_limit = np.percentile(data1['sqft_lot'],25) - 0.5*iqr
# sns.distplot(data1[(data1['sqft_lot']>lower_limit) & (data1['sqft_lot']<upper_limit)]['sqft_lot'])
# print(len(gk_log) - len(data1[(data1['sqft_lot']>lower_limit) & (data1['sqft_lot']<upper_limit)]['sqft_lot']))
# plt.show()

In [ ]:
plotting some of the price correlations here

In [ ]:
sns.regplot(x = 'sqft_living', y = 'price', data = data2);

In [ ]:
sns.regplot(x = 'sqft_lot', y = 'price', data = data2);

In [ ]:
sns.regplot(x = 'sqft_above', y = 'price', data = data2);

In [ ]:
sns.regplot(x = 'sqft_basement', y = 'price', data = data2);

separate my Y axis and then drop that column from my variables data
numericals & categoricals

In [ ]:
Y = data2['price']

In [ ]:
X = data2.drop(['price'], axis=1)

In [ ]:
x_num = data2.select_dtypes(np.number)
x_cat = data2.select_dtypes(np.object)

In [ ]:
x_num.head()

In [ ]:
x_cat.head()

scaling numericals here

In [ ]:
transformer = MinMaxScaler().fit(x_num)
x_min_max = transformer.transform(x_num)
x_min_max

transform & encode our data, setting up for linear regression models
using 3 different train-test sizes
metrics (r squared, mean squared error)

In [ ]:
transformer = Normalizer().fit(x_num)
x_normalized = transformer.transform(x_num)
encoder = OneHotEncoder().fit(x_cat)
encoded = encoder.transform(x_cat).toarray()
X = np.concatenate((x_normalized, encoded), axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 100)

In [ ]:
X_train.shape

In [ ]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
predictions  = lm.predict(X_test)
print(r2_score(y_test, predictions), mse(y_test, predictions))

scaling for knn  model here

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors = 6)  
model.fit(X_train, y_train)

predictions = model.predict(X_test)
score = model.score(X_test, y_test)

lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
r2_score(y_test, predictions)

first alternative here for benchmark(test_size = 0.4 random_state = 100)
r2_score - 0.7218125968088861   mse - 36040260132.244644
knn - 0.791567956216187 (n = 10) 

2nd alternative results for benchmark(test_size = 0.3 random_state = 100)
r2_score - 0.7213489908608615   mse - 35901959754.57134
knn alternatives - 0.8047915490188631 (n =10)

3rd alternative results for benchmark(test_size = 0.2 random_state = 100)
r2_score - 0.745428624242114   mse - 31914397687.52856
knn alternatives - 0.8201692422906726 (n = 6)

bias over fitting

In [ ]:
#from sklearn.neighbors import KNeighborsRegressor
scores = []
for i in range(2, 10):
    model = KNeighborsRegressor(n_neighbors=i)
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))


plt.figure(figsize=(10,6))
plt.plot(range(2, 10),scores,color = 'blue',linestyle='dashed', 
         marker='o',markerfacecolor='red', markersize=10)
plt.title('accuracy scores vs. K Value')
plt.xlabel('K')
plt.ylabel('Accuracy')

Conclusion

After pre-processing and cleaning our first test had a result of 74%. We had a better performance with the KNN regression model, that we applied after pre-processing, using the MinMax Scaler, and cleanning the Data. This time we had an accuracy of 82% using K = 6.